In [1]:
#import libraries

import os
import sys
import subprocess
import argparse
import pandas as pd
import numpy as np
import geopandas as gpd


In [2]:
#import excel file
df_km = pd.read_excel("C:/Users/20204018/OneDrive - TU Eindhoven/Documents/Master DSAI/YEAR 1 Q4/Knowledge Engineering Project/Datasets/Regionale_klimaat_monitor_2023.xlsx", header=1)
#remove empty columns

df_km.dropna(axis=1, how='all', inplace=True) 

In [3]:
df_km.columns

Index(['Gemeente', 'Aantal geregistreerde personenauto's per inwoner',
       'Aantal geregistreerde personenauto's per huishouden',
       'Aantal geregistreerde personenauto's per woning',
       'Bevolking totaal [aantal]', 'Huishoudens [aantal]',
       'Gemiddeld inkomen per inwoner [€]',
       'Gemiddeld inkomen per huishouden [€]',
       'Woningdichtheid (aantal woningen per hectare)'],
      dtype='object')

In [4]:
df_provincie = pd.read_excel("C:/Users/20204018/OneDrive - TU Eindhoven/Documents/Master DSAI/YEAR 1 Q4/Knowledge Engineering Project/Datasets/provincie_gemeente.xlsx")


In [5]:
df_provincie = df_provincie[['gemeente', 'provincie']]

In [6]:
df_km.columns = (
    df_km.columns
    .str.strip()                   # remove leading/trailing whitespace
    .str.lower()                   # convert to lowercase
    .str.replace(' ', '_')         # replace spaces with underscores
    .str.replace(r'[^\w_]', '', regex=True)  # remove special characters (like € or brackets)
)

df_km.columns = df_km.columns.str.rstrip('_')  # remove trailing underscores

In [7]:
#join the two dataframes on the 'gemeente' column
df_km_merge = df_km.merge(df_provincie, left_on='gemeente', right_on='gemeente', how='left')




In [8]:
df_km_merge.isnull().sum()
#sprint the rows out of the missing values
missing_values = df_km_merge[df_km_merge.isnull().any(axis=1)]
missing_values

#create a mapping of the province names to their abbreviations


,gemeente,aantal_geregistreerde_personenautos_per_inwoner,aantal_geregistreerde_personenautos_per_huishouden,aantal_geregistreerde_personenautos_per_woning,bevolking_totaal_aantal,huishoudens_aantal,gemiddeld_inkomen_per_inwoner,gemiddeld_inkomen_per_huishouden,woningdichtheid_aantal_woningen_per_hectare,provincie
24,Beek,0.60,1.27,1.27,16132,7568,32800,69900,3.58,NaN
65,Den Haag,0.36,0.72,0.75,562839,281186,32900,65900,32.78,NaN
128,Hengelo,0.59,1.23,1.23,82311,39355,30500,63800,6.50,NaN
158,Laren,0.59,1.31,1.25,11712,5256,50300,112100,4.41,NaN
181,Middelburg,0.46,0.97,0.95,49956,23928,30800,64300,5.06,NaN
238,Rijswijk,0.43,0.90,0.90,57997,27995,33900,70200,20.13,NaN
265,Stein,0.60,1.29,1.26,24772,11607,31900,68100,5.65,NaN


In [9]:
manual_provincie_mapping = {
    'Beek': 'Limburg',
    'Den Haag': 'Zuid-Holland',
    'Hengelo': 'Overijssel',
    'Laren': 'Noord-Holland',
    'Middelburg': 'Zeeland',
    'Rijswijk': 'Zuid-Holland',
    'Stein': 'Limburg'
}

# Alleen missende provincies bijvullen met de mapping
mask = df_km_merge['provincie'].isna()
df_km_merge.loc[mask, 'provincie'] = df_km_merge.loc[mask, 'gemeente'].map(manual_provincie_mapping)


In [ ]:
#fill in the missing values in the columns of the income with the median of the province column

income_columns = ['gemiddeld_inkomen_per_inwoner', 
                  'gemiddeld_inkomen_per_huishouden']


for column in income_columns:
    df_km_merge[column] = df_km_merge[column].replace(0, np.nan)  # Replace 0 with NaN for median calculation

#fill missing values with the median of the province
for column in income_columns:
    df_km_merge[column] = df_km_merge.groupby('provincie')[column].transform(lambda x: x.fillna(x.median()))

